In [1]:
! pip install ripser persim

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.3/827.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 3.3 MB/s eta 0:00:00
  Created wheel for hopcroftkarp: filename=hopcroftkarp-1.2.5-py2.py3-none-any.whl size=18104 sha256=d887c145aa54bb67237e6c745b50687ee8ef896ae08fbef3ad3af4d36fbed046
  Stored in directory: /root/.cache/pip/wheels/2a/fd/fe/f4b8fd82894e1d9e04040ef41dc5ae6eb7a8e9b0ef5a9402fe
Successfully built hopcroftkarp


In [16]:
import ripser, persim, warnings
import numpy as np

## Function that returns bottleneck distance

In [2]:
def getdist(feature):
  '''
  Takes a 2D array with columns x,y
  '''
                  # Line has same width and number of X points as feature...          but with y values at zero
  line = np.array([np.linspace(min(feature[:,0]),max(feature[:,0]),len(feature[:,0])),np.zeros_like(feature[:,1])])
  feature_dgm = ripser.ripser(feature)['dgms'][0]
  line_dgm = ripser.ripser(line.T)['dgms'][0]
  return persim.bottleneck(feature_dgm, line_dgm)

## Function that loops over data and stores bottleneck distances in a higher order array

In [28]:
def sliding_windows(pointCloud, dx=None, maxWindow=None, minWindow=None, step=None):
  '''
  Accepts 2D snapshots
  TODO: accept 3D time series
  When this is done, each alongshore cell has a spectrum of bottleneck
    distances for a range of window sizes centered on those cells

  '''
  x,y = pointCloud.T
  xRange = x.max() - x.min()

  # Prefer input parameters, these are example defaults
  if dx is None:
    dx = np.diff(x)[np.diff(x)>0].min() # cell size
  # window widths
  if maxWindow is None:
    maxWindow = xRange
  if minWindow is None:
    minWindow = dx*10 # probably want 3
  if step is None:
    step = minWindow

  # I *think* these are correct... but don't sue me
  offsets = step*np.arange(1,maxWindow//step)/2
  positions = dx*np.arange(xRange//dx+1)+x.min()

  print(f"Positions: {positions}")
  print(f"Offsets: {offsets}")

  # Create array to hold bottleneck distances
  dists = np.zeros((len(positions), len(offsets)))

  # Since we have periodic boundary conditions, we need copy the point cloud on
  #  each end of the domain before looping over the original middle section
  lbuff, rbuff = pointCloud.copy(), pointCloud.copy()
  lbuff[:, 0] -= xRange
  rbuff[:, 0] += xRange
  buff = np.vstack((lbuff, pointCloud, rbuff))
  buffx,buffy = buff.T

  for posIdx,pos in enumerate(positions):
    print(pos)
    for offsetIdx,offset in enumerate(offsets):
      with warnings.catch_warnings():
        warnings.filterwarnings("ignore", message=r"dgm[12] has points with non-finite death times;ignoring those points")
        dists[posIdx,offsetIdx] = getdist(pointCloud[(x>pos-offset) & (x<pos+offset)])

  return dists

## Read in data

In [8]:
# Manually digitized natural features
spit = np.loadtxt('data/spit.txt')
cape = np.loadtxt('data/cape.txt')
inlet = np.loadtxt('data/inlet.txt')

# Video data
coasts = dict()
for fnum in range(1,6):
    fnBase = f"jgrf217-sup-000{fnum+1}-ms0{fnum}"
    readin = np.load(f"data/{fnBase}_noPonds.npz")
    coasts[fnBase] = [readin[key] for key in readin.keys()]
    readin.close()
videoNum = 0
timestep = 0
# Returns 2D point cloud x,y
data = coasts[list(coasts.keys())[videoNum]][timestep]

## Run sliding windows

In [30]:
sliding_windows(data, dx=100, maxWindow=100, minWindow=10)

Positions: [  0 100 200 300 400 500 600 700]
Offsets: [ 5. 10. 15. 20. 25. 30. 35. 40. 45.]
0
100
200
300
400
500
600
700


array([[0.        , 0.59603173, 0.53921354, 0.62254685, 0.5866273 ,
        0.56127238, 0.54241866, 0.56638747, 0.55146843],
       [0.        , 0.        , 0.        , 0.41421354, 0.45421356,
        0.4475469 , 0.44278497, 0.47445452, 0.46797699],
       [0.        , 0.55707067, 0.51098776, 0.48738426, 0.47303706,
        0.49357861, 0.48270667, 0.51766181, 0.52532464],
       [0.        , 0.        , 0.4808802 , 0.46421355, 0.45421356,
        0.4475469 , 0.46976912, 0.47445452, 0.48789775],
       [0.        , 0.41421354, 0.53921354, 0.5304926 , 0.54148626,
        0.5485419 , 0.53109664, 0.51766181, 0.52532464],
       [0.70710677, 1.        , 0.61421353, 0.62254685, 0.5866273 ,
        0.56127238, 0.54241866, 0.52784991, 0.51625437],
       [0.52532464, 0.4668451 , 0.51098776, 0.5304926 , 1.        ,
        0.52190584, 0.50754684, 0.51766181, 0.52532464],
       [0.        , 0.51421356, 0.56572866, 0.5881266 , 0.5866273 ,
        0.57363385, 0.58494526, 0.57550389, 0.57611829]])

## Plot Heatmaps